In [24]:
import warnings
warnings.filterwarnings('ignore')

# Sklearn

## sklearn.grid_search

In [1]:
from sklearn import model_selection, datasets, linear_model, metrics

import numpy as np
import pandas as pd

### Генерация датасета

In [17]:
iris = datasets.load_iris()

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target,
                                                                                   test_size = 0.3, random_state = 0)

### Задание модели

In [3]:
classifier = linear_model.SGDClassifier(random_state= 0, tol=1e-3 )

### Генерация сетки

In [12]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [20]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter' : np.arange(5, 10),
    'alpha' : np.linspace(0.0001, 0.001, num=5),
}

In [21]:
cv = model_selection.StratifiedShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 0 )

### Подбор параметров и оценка качества

#### Grid search

In [22]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [25]:
%%time
grid_cv.fit(train_data, train_labels)

Wall time: 6.4 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
             error_score=nan,
             estimator=SGDClassifier(alpha=0.0001, average=False,
                                     class_weight=None, early_stopping=False,
                                     epsilon=0.1, eta0=0.0, fit_intercept=True,
                                     l1_ratio=0.15, learning_rate='optimal',
                                     loss='hinge', max_iter=1000,
                                     n_iter_no_change=5, n_jobs=None,
                                     penalty='l2...
                                     shuffle=True, tol=0.001,
                                     validation_fraction=0.1, verbose=0,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.001   ]),
                         'loss': ['hinge

In [26]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=9,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=0, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [27]:
print(grid_cv.best_score_)
print(grid_cv.best_params_)

0.9047619047619048
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


In [33]:
grid_cv.cv_results_

{'mean_fit_time': array([0.00349774, 0.00309825, 0.00259919, 0.00269833, 0.00229864,
        0.00289834, 0.00259831, 0.00239832, 0.00219831, 0.00299833,
        0.00239854, 0.00219858, 0.00229852, 0.00229864, 0.00239859,
        0.00229843, 0.00219848, 0.00259843, 0.00229869, 0.00239847,
        0.00209858, 0.00269825, 0.00229867, 0.00239873, 0.0023983 ,
        0.00229867, 0.00219891, 0.00209837, 0.00259867, 0.00209889,
        0.00229874, 0.00209873, 0.00249844, 0.00259855, 0.00269868,
        0.00249829, 0.00229847, 0.0022989 , 0.00229833, 0.00259805,
        0.00249834, 0.00219877, 0.00199878, 0.0021986 , 0.0024982 ,
        0.00229704, 0.00219874, 0.00229855, 0.00239837, 0.00209875,
        0.00240002, 0.00209904, 0.00239861, 0.00239921, 0.00259702,
        0.0022985 , 0.00229859, 0.00259826, 0.00229831, 0.00229852,
        0.00219851, 0.00199888, 0.00209882, 0.00239856, 0.00239871,
        0.00219867, 0.00249832, 0.00229862, 0.0020988 , 0.00239873,
        0.00229836, 0.00239825,

#### Randomized grid search

In [34]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_iter = 20,
                                                        random_state = 0)

In [35]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

Wall time: 717 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
                   error_score=nan,
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal',
                                           loss='hinge', max_iter=1000,
                                           n_iter_no_change=5, n_jobs=None,
                                           penal...
                                           verbose=0, warm_start=False),
                   iid='deprecated', n_iter=20, n_jobs=None,
                   param_distributions={'alpha': array([0.0001  , 0.000325, 0.00055 , 0.000775, 0.00

In [38]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.8666666666666666
{'penalty': 'l1', 'max_iter': 8, 'loss': 'squared_hinge', 'alpha': 0.0007750000000000001}
